In [1]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
arcpy.env.overwriteOutput = True

workspace = r"D:\\D_IndianaRisk\\data\\"
arcpy.env.workspace = workspace


In [2]:
#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = r"D:\\D_IndianaRisk\\data\\buildings_project.shp"

#set source for streams data
streams = r"D:\\D_IndianaRisk\\data\\streams_project.shp"

#set source for subwatersheds
all_subwatersheds = r"D:\\D_IndianaRisk\\data\\HUCs\\June_6_HUCS.shp"

In [3]:
#get distance from residential buildings to streams
#project buildings and streams so they are in the same GCS
sr = arcpy.SpatialReference(4269)

arcpy.Project_management(streams, workspace + "streams_project.shp", sr)


b_copy = arcpy.CopyFeatures_management(building_footprints, workspace + "buildings_copy")


arcpy.Project_management(b_copy, workspace+"buildings_project.shp", sr)



ExecuteError: ERROR 000229: Cannot open D:\\D_IndianaRisk\\data\\streams_project.shp
Failed to execute (Project).


In [ ]:
# #use near tool to get distance to steams
arcpy.Near_analysis(workspace+"buildings_project.shp",
                    workspace + "streams_project.shp",
                    method="GEODESIC")



In [ ]:
subwatershed_list = [] 
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []


In [ ]:
#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)
        
        print(subwatershed_number, " is the subwatershed to be worked on")

        #select subwatershed from shapefile with all of them
        s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                "HUC12 = '{}'".format(subwatershed_number))
        
        arcpy.CopyFeatures_management(s, "s_lyr.shp")
        subwatershed_selection1 = workspace + 's_lyr.shp'
        
        arcpy.Clip_analysis(workspace+"buildings_project.shp", 
                            subwatershed_selection1,
                            f"{subwatershed_number}_buildings.shp")

        #get all the distances in to the streams
        dist_to_stream_list = []
        with arcpy.da.SearchCursor(workspace+f"{subwatershed_number}_buildings.shp", ["NEAR_DIST"]) as distCursor: 
            for dist in distCursor:  
                dist_to_stream = dist[0]  
                dist_to_stream_list.append(dist_to_stream)  

        dist_to_stream_avg = mean(dist_to_stream_list)

        dist_to_stream_stdev = std(dist_to_stream_list)

        # print(dist_to_stream_avg)
        # print(dist_to_stream_stdev)

        dist_to_stream_avg_list.append(dist_to_stream_avg)
        dist_to_stream_stdev_list.append(dist_to_stream_stdev)

In [ ]:
st = time()
outputs = {'subwatershed': subwatershed_list,
           'dist_to_stream_avg (m)': dist_to_stream_avg_list,
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list}
           
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)'])
                                              
                                              
print(outputs_df)

outputs_df.to_excel(workspace+ 'last30_stream_dist.xlsx')
print(round(((time()-st)/60), 2) , 'minutes to process.')                                              

In [ ]:
print(dist_to_stream_avg)
print(dist_to_stream_stdev)

In [ ]:
workspace
